# Sequential Chats and Customer Onboarding

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

In [2]:
from autogen import ConversableAgent

## Creating the needed agents

In [3]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [4]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [5]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [6]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, we craft a series of tasks to facilitate the onboarding process.

In [7]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

In [8]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


C:\Users\SEMA Inc\Desktop\GITHUB3\Autogen1\venv\Lib\site-packages\autogen\agentchat\chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  name is Patrick Attankurugu


customer_proxy_agent (to Onboarding Personal Information Agent):

name is Patrick Attankurugu

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you for providing your name. Could you also tell me your location?

--------------------------------------------------------------------------------


Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  I live at Accra Digital Centre


customer_proxy_agent (to Onboarding Personal Information Agent):

I live at Accra Digital Centre

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{
  "name": "Patrick Attankurugu",
  "location": "Accra Digital Centre"
}

--------------------------------------------------------------------------------


Provide feedback to Onboarding Topic preference Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Autonomous Agents


customer_proxy_agent (to Onboarding Topic preference Agent):

Autonomous Agents

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to Customer Engagement Agent):

Let's find something fun to read.
Context: 
{
  "name": "Patrick Attankurugu",
  "location": "Accra Digital Centre"
}
Patrick Attankurugu is located in the Accra Digital Centre and is interested in reading about Autonomous Agents.

--------------------------------------------------------------------------------
Customer Engagement Agent (to customer_proxy_agent):

Hey Patrick from Accra Digital Centre! Did you know that the concept of Autonomous Agents actually has roots in science fiction? The idea of machines working independently and making decisions on their own can be traced back to e

## Print out the summary

In [9]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Patrick Attankurugu",
  "location": "Accra Digital Centre"
}


Patrick Attankurugu is located in the Accra Digital Centre and is interested in reading about Autonomous Agents.


Patrick from Accra Digital Centre is interested in reading about Autonomous Agents. The concept has roots in science fiction and is now a reality in various industries like robotics and AI. It's fascinating to see how technology is advancing rapidly.




## Print out the cost


In [10]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.000156, 'gpt-3.5-turbo-0125': {'cost': 0.000156, 'prompt_tokens': 198, 'completion_tokens': 38, 'total_tokens': 236}}, 'usage_excluding_cached_inference': {'total_cost': 0.000156, 'gpt-3.5-turbo-0125': {'cost': 0.000156, 'prompt_tokens': 198, 'completion_tokens': 38, 'total_tokens': 236}}}


{'usage_including_cached_inference': {'total_cost': 7.05e-05, 'gpt-3.5-turbo-0125': {'cost': 7.05e-05, 'prompt_tokens': 75, 'completion_tokens': 22, 'total_tokens': 97}}, 'usage_excluding_cached_inference': {'total_cost': 7.05e-05, 'gpt-3.5-turbo-0125': {'cost': 7.05e-05, 'prompt_tokens': 75, 'completion_tokens': 22, 'total_tokens': 97}}}


{'usage_including_cached_inference': {'total_cost': 0.00048650000000000006, 'gpt-3.5-turbo-0125': {'cost': 0.00048650000000000006, 'prompt_tokens': 373, 'completion_tokens': 200, 'total_tokens': 573}}, 'usage_excluding_cached_inference': {'total_cost': 0.00048650000000000006, 'gpt-3.5-turbo-0125': {'cost': 0.